## Calculate the variance of the error for GLV

Since $\mathbb{E}[err] = 0$, we have 

> $ \mathrm{Var}(err) = \mathbb{E}[err^2]$. 

with the error

> $ err = 1 + \frac{\alpha}{x_\mathrm{eff}\beta_\mathrm{eff}} = 1 - \frac{n}{d}$, 

where

> $ n := \sum_{ijkl} A_{ij}A_{kl}$ ,

> $ d := \sum_{ijk} A_{ij}A_{jk}$ ,

and the $A_{ij}$ are iid entreis of the weight matrix $A$, drawn from a distribution with mean $\mu$ and variance $\sigma$.

Inserting into the variance, we get

$\mathrm{Var}(err) = \mathbb{E}\frac{n^2 - 2nd + d^2}{d^2} \approx \frac{\mathbb{E}[n^2 - 2nd + d^2]}{\mathbb{E}[d^2]}$ .

The approximation is expected to hold for large networks with non-zero mean for $A_{ij}$.

By automatizing the combinatorial problems associated with the sums, I can show that 

> $\frac{1}{S^2} \mathbb{E}[n^2 - 2nd + d^2]  = \left(S^3 - S^2 - 3S + 3 \right) \sigma^4 + (S - 1) \mu_4 $ 

and 

> $\frac{1}{S^2} \mathbb{E}[d^2]  = S^6 \mu^4 + 6 S^4 \sigma^2 \mu^2 + \left(S^3 + 2S^2 - 3S\right) \sigma^4 + 4 S^2 \mu_3 \mu + S \mu_4 $ ,

where $\mu_3$ and $\mu_4$ are the centered $3^{rd}$ and $4^{th}$ moment ('skewness' and 'kurtosis').

Making the approximation indicated above, we thus get

> $\mathrm{Var}(err) \approx \frac{\mathbb{E}[n^2 - 2nd + d^2]}{\mathbb{E}[d^2]} \sim \frac{1}{S^3 \left(\frac{\mu}{\sigma}\right)^4}$ , 

or in terms of a necessary condition to have small variance on the error:

> $S \gg \left(\frac{\mu}{\sigma}\right)^\frac{4}{3}$ . 
    


In [1]:
from __future__ import print_function
import numpy as np
import itertools as it
import sympy


## Define functions

In [2]:
def generate_consistent_index_matrices(dim):
    """ Generate all possible iterations (only upper triangular matrices) 
    and throw away all inconsistent ones. If matrix is kept, set diagonal elements to `True`.
    """
    # Make sure your not filling the memory completely...
    assert dim <= 7, "Larger dimension take too long."
    # Number of entries in upper triangular matrix without diagonal
    red_dim = dim * (dim - 1) / 2
    # Iterate over all possible realizations
    Ps = []
    for entries in it.product([True, False], repeat = red_dim):
        P = np.zeros((dim, dim), dtype=bool)
        for i in range(dim - 1):
            si = sum(range(dim - 1 - i, dim - 1))
            P[i, i+1:] = entries[i + si: si + dim - 1]
    
        # Throw away inconsistent ones
        throw_away = False
        for x in range(dim-1):
            for y in range(x + 1, dim):
                for z in range(y + 1, dim):
                    if P[x, y] or P[x, z]:
                        if (P[x, y] and P[x, z]) != P[y, z]:
                            throw_away = True
                            break
                if throw_away:
                    break
            if throw_away:
                break
        if not throw_away:
            np.fill_diagonal(P, True)
            Ps.append(P)
            
    return Ps

def calc_cliques(Ps):
    sorted_clique_lengths = []
    m_Ps = np.zeros(len(Ps), dtype=int)
    
    dim_P = len(Ps[0])

    for j, P in enumerate(Ps):
        indices = set(range(dim_P))
        
        # Initial clique containing the first index
        cliques = [set([y for y, P_0_y in enumerate(P[0]) if P_0_y])]

        # Form other cliques
        for i in range(dim_P):
            if set.union(*cliques) == indices:
                break
            else:
                idx = min(indices - set.union(*cliques))
                P_idx = P[idx, idx: ]
                cliques.append(set([y + idx for y, P_idx_y in enumerate(P_idx) if P_idx_y]))

        # Collapse cliques
        sorted_clique_len = np.sort([len(clique) for clique in cliques])[::-1]
        sorted_clique_lengths.append(tuple(sorted_clique_len))
        m_Ps[j] = len(sorted_clique_len)
    
    return sorted_clique_lengths, m_Ps
        
def collapse_to_4d(Ps):
    dim_P = len(Ps[0])
    dim_R = 4
    Rs = np.zeros((len(Ps), dim_R, dim_R), dtype=bool)
    
    
    if dim_P == 6:
        # For denominator: collapse 6d to 4d
        for j, P in enumerate(Ps):
            R = np.zeros((dim_R, dim_R), dtype=bool)

            R[0, 1] = P[0, 4] and P[1, 4] #R[1, 1]
            R[0, 2] = P[0, 2] and P[4, 5] #R[0, 2]
            R[0, 3] = P[0, 5] and P[3, 4] #R[0, 3]
            R[1, 2] = P[2, 4] and P[1, 5] #R[1, 2]
            R[1, 3] = P[4, 5] and P[1, 3] #R[1, 3]
            R[2, 3] = P[2, 5] and P[3, 5] #R[2, 3]

            np.fill_diagonal(R, True)
            Rs[j] = R
    
    if dim_P == 7:
        # For den * num term: collapse 7d to 4d
        for j, P in enumerate(Ps):
            R = np.zeros((dim_R, dim_R), dtype=bool)

            R[0, 1] = P[0, 2] and P[1, 3] #R[1, 1]
            R[0, 2] = P[0, 4] and P[1, 6] #R[0, 2]
            R[0, 3] = P[0, 6] and P[1, 5] #R[0, 3]
            R[1, 2] = P[2, 4] and P[3, 6] #R[1, 2]
            R[1, 3] = P[2, 6] and P[3, 5] #R[1, 3]
            R[2, 3] = P[4, 6] and P[5, 6] #R[2, 3]

            np.fill_diagonal(R, True)
            Rs[j] = R
            
    return Rs
    
def get_unique_cliques_and_multiplicities(sorted_clique_lengths, m_Ps, dim_P):
    unique_scls = set(sorted_clique_lengths)
    list_scl_m_c = []
    m_factors_all = []
    for unique_scl in unique_scls:
        # Indices where the unique sorted clique length equals that of P in the list Ps
        idx = [j for j, scl in enumerate(sorted_clique_lengths) if scl == unique_scl]

        # Get the number of appearence of multiplicity factors S, S-1, S-2, ..., S-(dim_P-1)
        unique_m_P, counts = np.unique(m_Ps[idx], return_counts=True)
        m_factors = np.zeros(dim_P, dtype=int)
        m_factors[unique_m_P - 1] = counts

        # Append to a list containing the sored clique lengths and corresponding multiplicity factors
        list_scl_m_c.append([unique_scl, m_factors])
    #     print(unique_scl, m_factors)
        m_factors_all.append(m_factors)
    
    m_factors_all = np.asarray(m_factors_all)
    
    # Sort multiplicity factors
    m_factors_all = m_factors_all[np.argsort(list(unique_scls))[::-1]]
    return list_scl_m_c, m_factors_all

def print_results(list_scl_m_c):
    """ Print the results in a nice way (to understand what's going on).
    """
    dim_P = len(list_scl_m_c[0][1])

    names                           = ['X^4    ','X^3 Y  ', 'X^2 Y^2', 'X^2 Y Z', 'X Y Z U',    ]
    corresponding_clique_lengths    = [(4,),     (3, 1),    (2, 2),    (2, 1, 1), (1, 1, 1, 1), ]

    print(" "*11 + "Multipl. number m_P")
    m_p_str_title = ""
    for i in range(dim_P):
        m_p_str_title += "%5d"%(i+1)
    print("Cliques  " + m_p_str_title + " "*5 + " tuple")
    print("-"*60)

    for name, cl in zip(names, corresponding_clique_lengths):
        for scl_m_c in list_scl_m_c:
            if scl_m_c[0] == cl:
                m_str = ""
                for m_c in scl_m_c[1]:
                    m_str += "%5d"%m_c
                print(name + "  " + m_str + " "*5, scl_m_c[0])
    

def calc_expected_value(m_factors_all, S):
    dim_P = m_factors_all.shape[1]
    j = sympy.symbols('j', integer=True)

    # Check-sum
    sum_S = 0
    for i in range(dim_P):
        prod_S = sympy.product(S-j, (j, 0, i))
        sum_S += prod_S * np.sum(m_factors_all, axis=0)[i]
    
    if S**dim_P == sympy.expand(sum_S):
        print("Checksum = ", sympy.expand(sum_S), "is ok!") # == S**6???

    # Calculate factors for expected value
    a = []
    for k in range(len(m_factors_all)):
        sum_S = 0
        for i in range(dim_P):
            prod_S = sympy.product(S-j, (j, 0, i))
            sum_S += prod_S * m_factors_all[k, i]
        a.append(sympy.expand(sum_S))
        
    print("\nMultiplicity factors:")
    for i, ai in enumerate(a):
        print("a_%d = "%i, ai)

    # Print out the entire term
    mu4, mu3, mu2, mu = sympy.symbols('mu4 mu3 mu2 mu')
    
    expected_value = (mu4         * a[0] + 
                      mu3 * mu    * (4*a[0] + a[1]) + 
                      mu2 * mu**2 * (6*a[0] + 3*a[1] + 2*a[2] + a[3]) + 
                      mu2**2      * a[2] + 
                      mu**4       * sum(a))
    
    return expected_value


## Calculate expected values of $n^2$, $d^2$ and $n \cdot d$.

In [3]:
# Symbolic variable for network size
S = sympy.symbols('S')


In [4]:
# Calculate n^2 term
# Reduced to 4d, but running over $M = S^2$ entries. 
print("Calculate n^2 term\n")

dim = 4

Ps                          = generate_consistent_index_matrices(dim)

sorted_clique_lengths, m_Ps = calc_cliques(Ps)
        
list_scl_m_c, m_factors_all = get_unique_cliques_and_multiplicities(sorted_clique_lengths, m_Ps, dim)

print_results(list_scl_m_c)
    
M = S**2
E_n_sq = calc_expected_value(m_factors_all, M)

print("\nE[n^2]:")
print(sympy.expand(E_n_sq))

Calculate n^2 term

           Multipl. number m_P
Cliques      1    2    3    4      tuple
------------------------------------------------------------
X^4          1    0    0    0      (4,)
X^3 Y        0    4    0    0      (3, 1)
X^2 Y^2      0    3    0    0      (2, 2)
X^2 Y Z      0    0    6    0      (2, 1, 1)
X Y Z U      0    0    0    1      (1, 1, 1, 1)
Checksum =  S**8 is ok!

Multiplicity factors:
a_0 =  S**2
a_1 =  4*S**4 - 4*S**2
a_2 =  3*S**4 - 3*S**2
a_3 =  6*S**6 - 18*S**4 + 12*S**2
a_4 =  S**8 - 6*S**6 + 11*S**4 - 6*S**2

E[n^2]:
S**8*mu**4 + 6*S**6*mu**2*mu2 + 4*S**4*mu*mu3 + 3*S**4*mu2**2 - 3*S**2*mu2**2 + S**2*mu4


In [5]:
# Calculate d^2 term
print("Calculate n^2 term\n")

dim = 6

Ps                          = generate_consistent_index_matrices(dim)

_, m_Ps                     = calc_cliques(Ps)
        
Rs                          = collapse_to_4d(Ps)
    
sorted_clique_lengths, _    = calc_cliques(Rs)

list_scl_m_c, m_factors_all = get_unique_cliques_and_multiplicities(sorted_clique_lengths, m_Ps, dim)

print_results(list_scl_m_c)
    
E_d_sq = S**2 * calc_expected_value(m_factors_all, S)

print("\nE[d^2]:")
print(sympy.expand(E_d_sq))

Calculate n^2 term

           Multipl. number m_P
Cliques      1    2    3    4    5    6      tuple
------------------------------------------------------------
X^4          1    0    0    0    0    0      (4,)
X^3 Y        0    4    0    0    0    0      (3, 1)
X^2 Y^2      0    5    1    0    0    0      (2, 2)
X^2 Y Z      0   20   34    6    0    0      (2, 1, 1)
X Y Z U      0    2   55   59   15    1      (1, 1, 1, 1)
Checksum =  S**6 is ok!

Multiplicity factors:
a_0 =  S
a_1 =  4*S**2 - 4*S
a_2 =  S**3 + 2*S**2 - 3*S
a_3 =  6*S**4 - 2*S**3 - 16*S**2 + 12*S
a_4 =  S**6 - 6*S**4 + S**3 + 10*S**2 - 6*S

E[d^2]:
S**8*mu**4 + 6*S**6*mu**2*mu2 + S**5*mu2**2 + 4*S**4*mu*mu3 + 2*S**4*mu2**2 - 3*S**3*mu2**2 + S**3*mu4


In [6]:
# Calculate n*d term
print("Calculate n^2 term\n")

dim = 7

Ps                          = generate_consistent_index_matrices(dim)

_, m_Ps                     = calc_cliques(Ps)

Rs                          = collapse_to_4d(Ps)
    
sorted_clique_lengths, _    = calc_cliques(Rs)

list_scl_m_c, m_factors_all = get_unique_cliques_and_multiplicities(sorted_clique_lengths, m_Ps, dim)

print_results(list_scl_m_c)
    
E_nd = S * calc_expected_value(m_factors_all, S)

print("\nE[n*d]:")
print(sympy.expand(E_nd))

Calculate n^2 term

1
2
3
4
5
           Multipl. number m_P
Cliques      1    2    3    4    5    6    7      tuple
------------------------------------------------------------
X^4          1    0    0    0    0    0    0      (4,)
X^3 Y        0   12    4    0    0    0    0      (3, 1)
X^2 Y^2      0    9    3    0    0    0    0      (2, 2)
X^2 Y Z      0   36  132   60    6    0    0      (2, 1, 1)
X Y Z U      0    6  162  290  134   21    1      (1, 1, 1, 1)
Checksum =  S**7 is ok!

Multiplicity factors:
a_0 =  S
a_1 =  4*S**3 - 4*S
a_2 =  3*S**3 - 3*S
a_3 =  6*S**5 - 18*S**3 + 12*S
a_4 =  S**7 - 6*S**5 + 11*S**3 - 6*S

E[n*d]:
S**8*mu**4 + 6*S**6*mu**2*mu2 + 4*S**4*mu*mu3 + 3*S**4*mu2**2 - 3*S**2*mu2**2 + S**2*mu4


## Put everything together to get $\mathbb{E}[err^2] \approx \frac{\mathbb{E}[n^2 - 2nd + d^2]}{\mathbb{E}[d^2]}$

In [7]:
# Putting everything together
print("\nE[n^2]:")
print(sympy.expand(E_n_sq))

print("\nE[d^2]:")
print(sympy.expand(E_d_sq))

print("\nE[n*d]:")
print(sympy.expand(E_nd))

print("\nE[n^2 - 2*n*d + d^2] / S**2:")
print(sympy.expand((E_n_sq - 2 * E_nd + E_d_sq) / S**2))

print("\nE[d^2] / S**2:")
print(sympy.expand(E_d_sq / S**2))


E[n^2]:
S**8*mu**4 + 6*S**6*mu**2*mu2 + 4*S**4*mu*mu3 + 3*S**4*mu2**2 - 3*S**2*mu2**2 + S**2*mu4

E[d^2]:
S**8*mu**4 + 6*S**6*mu**2*mu2 + S**5*mu2**2 + 4*S**4*mu*mu3 + 2*S**4*mu2**2 - 3*S**3*mu2**2 + S**3*mu4

E[n*d]:
S**8*mu**4 + 6*S**6*mu**2*mu2 + 4*S**4*mu*mu3 + 3*S**4*mu2**2 - 3*S**2*mu2**2 + S**2*mu4

E[n^2 - 2*n*d + d^2] / S**2:
S**3*mu2**2 - S**2*mu2**2 - 3*S*mu2**2 + S*mu4 + 3*mu2**2 - mu4

E[d^2] / S**2:
S**6*mu**4 + 6*S**4*mu**2*mu2 + S**3*mu2**2 + 4*S**2*mu*mu3 + 2*S**2*mu2**2 - 3*S*mu2**2 + S*mu4
